In [4]:
import requests
import pandas as pd
from config import abuse_API_key

res = requests.get("https://api.abuseipdb.com/api/v2/blacklist", headers={"Key": abuse_API_key})

{'meta': {'generatedAt': '2023-05-13T13:19:28+00:00'},
 'data': [{'ipAddress': '103.67.163.178',
   'countryCode': 'VN',
   'abuseConfidenceScore': 100,
   'lastReportedAt': '2023-05-13T13:17:02+00:00'},
  {'ipAddress': '47.108.59.43',
   'countryCode': 'CN',
   'abuseConfidenceScore': 100,
   'lastReportedAt': '2023-05-13T13:17:02+00:00'},
  {'ipAddress': '148.72.209.121',
   'countryCode': 'SG',
   'abuseConfidenceScore': 100,
   'lastReportedAt': '2023-05-13T13:17:01+00:00'},
  {'ipAddress': '139.59.108.166',
   'countryCode': 'SG',
   'abuseConfidenceScore': 100,
   'lastReportedAt': '2023-05-13T13:17:01+00:00'},
  {'ipAddress': '94.102.61.2',
   'countryCode': 'NL',
   'abuseConfidenceScore': 100,
   'lastReportedAt': '2023-05-13T13:17:01+00:00'},
  {'ipAddress': '92.50.179.236',
   'countryCode': 'RU',
   'abuseConfidenceScore': 100,
   'lastReportedAt': '2023-05-13T13:17:01+00:00'},
  {'ipAddress': '27.131.36.170',
   'countryCode': 'MY',
   'abuseConfidenceScore': 100,
   'last

In [5]:
data = res.json()
ips = []
for i in range (100):
    ips.append(data['data'][i]['ipAddress'])


In [6]:
detailed = []
i = 0
for address in ips:
    detailed.append(requests.get(f"https://api.abuseipdb.com/api/v2/check?ipAddress={address}",
             headers={"Key": "856a7889e9bc917f2b54aff296d3380cacdea7c240c3a106ae72f0c01d3d1c2d5c1ebdf7cb6e30c2"}).json())



{'errors': [{'detail': 'Daily rate limit of 1000 requests exceeded for this endpoint. See headers for additional details.',
   'status': 429}]}

In [23]:
isp_count = {}
i = 0
for entry in detailed:
    isp_count[entry['data']['domain']] = isp_count.get(entry['data']['domain'], 0) + 1
    i += 1
    if i > 80:
        break
sorted_isps = sorted(isp_count, reverse=True, key= lambda x: isp_count[x])
counts = {}
for entry in sorted_isps:
    counts[entry] = isp_count[entry]

counts

{'digitalocean.com': 13,
 'chinatelecom.com.cn': 6,
 'tencent.com': 6,
 None: 2,
 'fiberxpress.net': 2,
 'cyber.casa': 2,
 'ovh.com': 2,
 'microsoft.com': 2,
 'google.com': 2,
 'aliyun.com': 1,
 'godaddy.com': 1,
 'ufanet.ru': 1,
 'mykris.net': 1,
 'tigo.net.bo': 1,
 'cable-giant.com.tw': 1,
 'arvan.ir': 1,
 'rhinocloud.uk': 1,
 'asiatech.ir': 1,
 'serverion.com': 1,
 'unifiedlayer.com': 1,
 'rackip.com': 1,
 'kt.com': 1,
 'sobralnet.com.br': 1,
 'hkt.com': 1,
 'vodafone.it': 1,
 'redeminastelecom.com.br': 1,
 'des.capital': 1,
 'shadowserver.org': 1,
 'telmiix.com.py': 1,
 'glesys.se': 1,
 'cyble.net': 1,
 'iam.net.ma': 1,
 'sioru.com': 1,
 'comcast.net': 1,
 'alicloud.com': 1,
 'psychz.net': 1,
 'm247.com': 1,
 'ethr.net': 1,
 'oracle.com': 1,
 'spectrum.com': 1,
 'bluecom.so': 1,
 'partpayam.com': 1,
 'netnoartelecom.com.br': 1,
 'axtel.mx': 1,
 'baidu.com': 1,
 'telia.com': 1,
 'zanyun.cn': 1,
 'cht.com.tw': 1,
 '0-1.ir': 1,
 'bthoster.com': 1,
 'fibernetms.com.br': 1,
 'azeronline

In [10]:
isp_count = {}
unique_count = {}
for entry in detailed:
    isp_count[entry['data']['domain']] = isp_count.get(entry['data']['domain'], 0) + 1
    unique_count[entry['data']['domain']] = unique_count.get(entry['data']['domain'], 0) + entry['data']['numDistinctUsers']

ratio = {}
for domain, count in isp_count.items():
    ratio[domain] = count/unique_count[domain]
    
sorted_ratio = dict(sorted(ratio.items(), key= lambda v: v[1], reverse=True))
sorted_ratio

sorted_unique = dict(sorted(unique_count.items(), key= lambda v: v[1]))
# sorted_unique

# users >> abusive IPs -> users are less abusive
# users ~= abusive IPs -> users are more abusive

# abuse rank higher than user rank -> -- (abusive IPs/users >> users)
# abuse rank lower than user rank -> ++ (abusive IPs/users << users)
# equal: insignificant

# ratio_rank = {}
# i = 1
# for e in sorted_ratio:
#     ratio_rank[e] = i
#     i += 1

# users_rank = {}
# i = 1
# for e in sorted_unique:
#     users_rank[e] = i
#     i += 1

# abusive_score = {}
# for domain in ratio_rank:
#     abusive_score[domain] = users_rank[domain] - ratio_rank[domain]
    
# abusive_score = dict(sorted(abusive_score.items(), key= lambda v: v[1], reverse=True))
# abusive_score

del sorted_ratio[None]
for domain in isp_count:
    if isp_count[domain] < 5:
        del sorted_ratio[domain]
df = pd.DataFrame.from_dict(sorted_ratio, orient='index', columns=['Abuse Index'])
df.index.name = "Domain"
df

KeyError: 'data'